In [2]:
import time
import pandas as pd

# Definições Iniciais

In [3]:
limite_valor_saque = 500
limite_n_saque = 3

### Tabela do Usuário e Conta

In [197]:
df_usuario = pd.DataFrame(columns=['IdUsuario', 'NomeUsuario', 'CpfUsuario'])
init_data = pd.DataFrame({
                'IdUsuario': [1],
                'NomeUsuario': ['Usuario1'],
                'CpfUsuario': ['11111111111']
            })
df_usuario = pd.concat([df_usuario, init_data], ignore_index=True)
df_usuario

,IdUsuario,NomeUsuario,CpfUsuario
0,1,Usuario1,11111111111


In [210]:
df_conta = pd.DataFrame(columns=['IdConta', 'IdUsuario', 'Agencia', 'NumeroConta'])
init_data = pd.DataFrame({
                'IdConta': [1],
                'IdUsuario': [1],
                'Agencia': ['0001'],
                'NumeroConta': ['11111']
            })
df_conta = pd.concat([df_conta, init_data], ignore_index=True)
df_conta

,IdConta,IdUsuario,Agencia,NumeroConta
0,1,1,0001,11111


### Tabela Referente a Conta do Usuário

In [211]:
valor_inicial = 2000

df_mov = pd.DataFrame(columns=['VinculoUsuario', 'VinculoConta', 'Valor_Conta', 'Data_Hora', 'Tipo_Operacao'])
init_data = pd.DataFrame({'VinculoUsuario': [1],
                          'VinculoConta': [1],
                            'Valor_Conta': [valor_inicial],
                            'Data_Hora': [time.strftime("%d/%m/%Y %H:%M")],
                            'Tipo_Operacao': ['I']})
df_mov = pd.concat([df_mov, init_data], ignore_index=True)
df_mov

,VinculoUsuario,VinculoConta,Valor_Conta,Data_Hora,Tipo_Operacao
0,1,1,2000,21/02/2024 10:39,I


# Definição das funções operacionais

### Cadastro Usuario

In [200]:
def valida_cpf(cpf):
    """
    Valida um CPF, retornando True se for válido e False caso contrário.
    Não será removido o caso de dígitos iguais, pois eles serão utilizados como exemplo
    """
    # Removendo caracteres não numéricos
    cpf = ''.join(filter(str.isdigit, cpf))
    
    # Verifica se o CPF tem 11 dígitos
    if len(cpf) != 11:
        return False

    for i in range(9, 11):
        valor = sum((int(cpf[num]) * ((i+1) - num) for num in range(0, i)))
        digito = ((valor * 10) % 11) % 10
        if str(digito) != cpf[i]:
            return False
    return True

def CadastraUsuario(df_cad, Nome, CPF):

    if CPF in df_cad['CpfUsuario'].values:
        print("Erro: CPF já cadastrado.")
        return df_cad

    if not valida_cpf(CPF):
        print("Erro: CPF inválido.")
        return df_cad
    novo_id = df_cad['IdUsuario'].max() + 1 if not df_cad.empty else 1
    UserData = pd.DataFrame({
        'IdUsuario': [novo_id],
        'NomeUsuario': [Nome],
        'CpfUsuario': [CPF]
    })
    # Atualização correta do DataFrame
    df_atualizado = pd.concat([df_cad, UserData], ignore_index=True)
    return df_atualizado

In [201]:
df_usuario = CadastraUsuario(df_usuario, 'Usuario2', '22222222222')
df_usuario = CadastraUsuario(df_usuario, 'Usuario3', '33333333333')
df_usuario

,IdUsuario,NomeUsuario,CpfUsuario
0,1,Usuario1,11111111111
1,2,Usuario2,22222222222
2,3,Usuario3,33333333333


### Cadastro Conta

In [204]:
def encontrar_id_conta(df_conta, Cpf, agencia, numero_conta):

    id_usuario = df_usuario.loc[df_usuario['CpfUsuario'] == Cpf, 'IdUsuario'].values[0]
    conta_filtrada = df_conta[
        (df_conta['IdUsuario'] == id_usuario) & 
        (df_conta['Agencia'] == agencia) & 
        (df_conta['NumeroConta'] == numero_conta)
    ]
    
    # Verificando se foi encontrada alguma conta que corresponda aos critérios
    if not conta_filtrada.empty:
        # Retornando o IdConta da primeira (e presumivelmente única) linha correspondente
        return conta_filtrada['IdConta'].iloc[0]
    else:
        # Caso não encontre uma conta que corresponda, retorna um valor indicativo, como None
        return None

encontrar_id_conta(df_conta, '33333333333', '0002', '33333')

4

In [207]:
def LocalizarId(df_usuario, CpfEntrada):
    Id = df_usuario.loc[df_usuario['CpfUsuario'] == CpfEntrada, 'IdUsuario'].values[0]
    return Id

def CadastraConta(df_cad, Cpf, Agencia, NumeroConta):

    IdUsuario = LocalizarId(df_usuario, Cpf)
    existencia = df_conta[(df_conta['Agencia'] == Agencia) & (df_conta['NumeroConta'] == NumeroConta)].any().any()

    if existencia:
        print("Erro: Agencia e Conta já Existe.")
        return df_cad

    if IdUsuario in df_cad['Agencia'].values:
        print("Erro: CPF já possui uma Conta.")
        return df_cad

    novo_id = df_cad['IdConta'].max() + 1 if not df_cad.empty else 1
    ContaData = pd.DataFrame({
        'IdConta': novo_id,
        'IdUsuario': [IdUsuario],
        'Agencia': [Agencia],
        'NumeroConta': [NumeroConta]
    })

    global df_mov
    init_data = pd.DataFrame({'VinculoUsuario': IdUsuario,
                              'VinculoConta': novo_id,
                                'Valor_Conta': [0],
                                'Data_Hora': [time.strftime("%d/%m/%Y %H:%M")],
                                'Tipo_Operacao': ['I']})

    df_mov = pd.concat([df_mov, init_data], ignore_index=True)
    df_mov

    df_atualizado = pd.concat([df_cad, ContaData], ignore_index=True)
    return df_atualizado

In [212]:
df_conta = CadastraConta(df_conta, '22222222222', '0001', '22222')
df_conta = CadastraConta(df_conta, '33333333333', '0001', '33333')
df_conta = CadastraConta(df_conta, '33333333333', '0002', '33333')
df_conta

,IdConta,IdUsuario,Agencia,NumeroConta
0,1,1,0001,11111
1,2,2,0001,22222
2,3,3,0001,33333
3,4,3,0002,33333


## Operação Depósito

In [215]:
def operacao_deposito(valor_deposito, Cpf, Agencia, NumeroConta, df):
    # Check if the input is numeric
    try:
        valor_deposito = float(valor_deposito)
    except ValueError:
        print("""
              O valor informado não é um número válido, favor tentar novamente
              ----------------------------------------------------------------------

              Importante:
              O presente sistema utiliza '.' como identificador decimal.
              Para realizar o depósito, insira apenas um único valor numérico.  
              Não utilize letras ou símbolos especiais.
        """)
        return df

    IdUsuario = LocalizarId(df_usuario, Cpf)
    IdConta = encontrar_id_conta(df_conta, Cpf, Agencia, NumeroConta)

    if valor_deposito > 0:
        current_datetime = time.strftime("%d/%m/%Y %H:%M")

        new_data = pd.DataFrame({'VinculoUsuario': [IdUsuario],
                                 'VinculoConta': IdConta,
                                 'Valor_Conta': [valor_deposito],
                                 'Data_Hora': [current_datetime],
                                 'Tipo_Operacao': ['D']})
        df = pd.concat([df, new_data], ignore_index=True)

        print(f'Depósito realizado em: {current_datetime}')
        print(f'Depósito no valor de R$ {valor_deposito:.2f} realizado com sucesso.')
    else:
        print('Não é possível realizar um depósito nulo ou com valor negativo.')

    return df

## Operação Saque

In [225]:
def operacao_saque(valor_saque, Cpf, Agencia, NumeroConta, df_transacoes):

    IdUsuario = LocalizarId(df_usuario, Cpf)
    IdConta = encontrar_id_conta(df_conta, Cpf, Agencia, NumeroConta)
    saldo_conta = df_mov[(df_mov['VinculoUsuario'] == IdUsuario) & (df_mov['VinculoConta'] == IdConta)]['Valor_Conta'].sum()

    if saldo_conta < valor_saque:
        print("Saldo insuficiente para realizar o saque.")
        return df_transacoes

    if valor_saque <= 0 or valor_saque > limite_valor_saque:
        print(f"Valor de saque inválido. O valor do saque não pode ser negativo ou maior do que R$ {limite_valor_saque:.2f}.")
        return df_transacoes

    data_hoje = time.strftime("%d/%m/%Y")
    saques_hoje = df_transacoes[(df_transacoes['Data_Hora'].str[:10] == data_hoje) & (df_transacoes['Tipo_Operacao'] == 'S')].shape[0]
    if saques_hoje >= limite_n_saque:
        print(f"Limite diário de saques atingido. Você já realizou {limite_n_saque:.0f} saques hoje.")
        return df_transacoes

    saldo_conta -= valor_saque
    current_datetime = time.strftime("%d/%m/%Y %H:%M")

    new_data = pd.DataFrame({'VinculoUsuario': [IdUsuario],
                                'VinculoConta': IdConta,
                                'Valor_Conta': [-valor_saque],
                                'Data_Hora': [current_datetime],
                                'Tipo_Operacao': ['S']})
    df_transacoes = pd.concat([df_transacoes, new_data], ignore_index=True)
    
    print(f'Saque realizado em: {current_datetime}')
    print(f'Saque no valor de R$ {valor_saque:.2f} realizado com sucesso.')

    return df_transacoes

## Operação Extrato

In [236]:
def operacao_extrato(Cpf, Agencia, NumeroConta, df_transacoes):

    IdUsuario = LocalizarId(df_usuario, Cpf)
    IdConta = encontrar_id_conta(df_conta, Cpf, Agencia, NumeroConta)

    filtro = (df_mov['VinculoUsuario'] == IdUsuario) & (df_mov['VinculoConta'] == IdConta)
    df_transacoes = df_transacoes[filtro]

    print("Extrato de transações:")
    for index, row in df_transacoes.iterrows():
        valor_transacao = row['Valor_Conta']
        if valor_transacao > 0:
            tipo_transacao = "Depósito"
        else:
            tipo_transacao = "Saque"
        print(f"{tipo_transacao}: R$ {abs(valor_transacao):.2f} - Data/Hora: {row['Data_Hora']}")

    saldo_conta = df_transacoes['Valor_Conta'].sum()
    print(f"\nSaldo atual da conta: R$ {saldo_conta:.2f}")